In [25]:
import pickle 
import pandas as pd 
import seaborn as sns
import numpy as np 
import json
import itertools
import lifelines
import glob
import os
import copy
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import gridspec
import matplotlib.cm as cm
from matplotlib.colors import Normalize, LinearSegmentedColormap
import matplotlib.patches as patches
from matplotlib.patches import Patch, Rectangle
from matplotlib.lines import Line2D
from lifelines import CoxPHFitter, KaplanMeierFitter
from lifelines.statistics import logrank_test
import sys
import scipy.stats

my_path = "/Users/jdf36/Desktop/bwh/cancer_regions/src/"

sys.path.insert(1, my_path)

from jenks import load_clinvar_df, getJenksBreaks, altered_generate_coding_position_list
from utils import place_position
from survival_tools import (
    annotate_region_diagram_ax, 
    get_km_curve_ax,
    risk_ratio,
    get_pfam_domains,
    get_single_gene_and_breaks,
    make_hr_lr_binary_diagram,
    get_colors,
    position_in_pfam_lists,
    get_pfam_domains,
    get_annotated_pfam_profiles,
    get_logrank_p,
    get_annotated_hr_lr_regions,
    get_pfam_comparison_logrank_vals
)
from annotate_funtional_assays import get_functional_from_key
from cosmic import get_cosmic_ratios, get_gene_cosmic, get_cosmic_ratios_altered
from config import gene_to_end_position 


In [26]:
def format_p_string(p):
    p_string = str(round(p, 4))
    if p < 0.0001:
        sci_notation_pieces = "{:.2e}".format(p).split("e")
        exponent = int(sci_notation_pieces[1])
        decimal = sci_notation_pieces[0]
        p_string =  "$" + decimal + "x10^{" + str(exponent) + "}" +  "$"
        return p_string
    return "$" + p_string + "$"



In [27]:
file = open("../gene_to_df_delet_removed_092922.pickle",'rb')
gene_to_df = pickle.load(file)
logrank_df = pd.read_csv("../all_logrank_df_092922.csv")
regression_results = pd.read_csv("../regression_results_delet_removed_092922.csv", index_col = 0)
file = open("../mega_results_checkppoint_092922.pickle",'rb')
mega_file = pickle.load(file)

In [28]:
all_func_data = []

In [29]:
gene_df = logrank_df.loc[logrank_df["gene"] == "BRCA1"]

for key in gene_df["key"].values:
    optimal = logrank_df.loc[logrank_df["key"] == key].iloc[0]

    df = pd.read_excel("../../data/functional_data/BRCA1_41586_2018_461_MOESM3_ESM.xlsx", skiprows = 2)
    df = df.loc[
        (~df["transcript_position"].str.contains("+", regex = False)) &
        (~df["transcript_position"].str.contains("-", regex = False)) &
        (df["consequence"] == "Missense")
    ]

    df["coding_position"] = df["transcript_position"].apply(lambda x : int(x))
    optimal = logrank_df.loc[logrank_df["key"] == key].iloc[0]
    optimal_breaks = list(map(lambda x : int(float(x)), optimal["breakpoints"].split("|")))
    optimal_high = list(map(lambda x : int(x), optimal["high_regions"].split("|")))
    optimal_low = list(map(lambda x : int(x), optimal["low_regions"].split("|")))
    reg = []
    for index, row in df.iterrows():
        p = row["coding_position"]
        region = place_position(optimal_breaks, p) + 1
        reg.append(region)

    df["region"] = reg
    col = "function.score.mean"
    c_pos_mean = {}
    for c in set(df["coding_position"].values):
        subset = df.loc[df["coding_position"] == c]
        c_pos_mean[c] = np.mean(subset[col].values)

    df["c_pos_mean"] = df["coding_position"].apply(lambda x : c_pos_mean[x])
    df = df.drop_duplicates(subset = "coding_position")
    df["in HR"] = df["region"].apply(lambda x : x in optimal_high)
    df["in LR"] = df["region"].apply(lambda x : x in optimal_low)
    hr_df = df.loc[df["in HR"]]
    lr_df = df.loc[df["in LR"]]

    if len(hr_df.loc[~hr_df[col].isna()]) > 0 and len(lr_df.loc[~lr_df[col].isna()]) > 0:
        a = hr_df.loc[~hr_df[col].isna()][col].values
        b = lr_df.loc[~lr_df[col].isna()][col].values
        p_test = scipy.stats.ks_2samp(a, b).pvalue
        all_func_data.append([key, p_test, len(a), len(b), ""])


In [30]:
gene_df = logrank_df.loc[logrank_df["gene"] == "BRCA2"]

for key in gene_df["key"].values:
    df_a = pd.read_excel('../../data/functional_data/Ikegami_BRCA2_41467_2020_16141_MOESM11_ESM.xlsx', index_col = 0, sheet_name = "Figure_3d")
    dict_a = {
        "Variant" : df_a.loc["Variant"].values,
        "Eta_average" : df_a.loc["Eta_average"].values
    }

    df_a = pd.DataFrame.from_dict(dict_a, orient = "index")
    df_a = df_a.transpose()
    pos = []
    ref = []
    alt = []
    for v in df_a["Variant"].values:
        if v in ["Wild-type","Empty-vector"]:
            pos.append(None)
            ref.append(None)
            alt.append(None)
        else:
            ref.append(v[0])
            alt.append(v[-1])
            pos.append(int(v[1:-1]))

    df_a["ref"] = ref
    df_a["alt"] = alt
    df_a["pos"] = pos
    df_a = df_a.dropna()
    df_a["pos"] = df_a["pos"].apply(lambda x : int(x))
    df_a["coding_pos"] = df_a["pos"].apply(lambda x : int(x * 3))

    c_pos_mean = {}
    for c in set(df_a["coding_pos"].values):
        subset = df_a.loc[df_a["coding_pos"] == c]
        c_pos_mean[c] = np.mean(subset["Eta_average"].values)

    df_a["c_pos_mean"] = df_a["coding_pos"].apply(lambda x : c_pos_mean[x])
    df_a = df_a.drop_duplicates(subset = "coding_pos")

    corr_dicts = {}
    results = []

    df = copy.deepcopy(df_a)
    optimal = logrank_df.loc[logrank_df["key"] == key].iloc[0]
    optimal_breaks = list(map(lambda x : int(float(x)), optimal["breakpoints"].split("|")))
    optimal_high = list(map(lambda x : int(x), optimal["high_regions"].split("|")))
    optimal_low = list(map(lambda x : int(x), optimal["low_regions"].split("|")))

    df = df.drop_duplicates(subset = "coding_pos")
    reg = []
    for index, row in df.iterrows():
        p = row["coding_pos"]
        region = place_position(optimal_breaks, p) + 1
        reg.append(region)

    df["region"] = reg
    df["in HR"] = df["region"].apply(lambda x : x in optimal_high)
    df["in LR"] = df["region"].apply(lambda x : x in optimal_low)
    hr_df = df.loc[df["in HR"]]
    lr_df = df.loc[df["in LR"]]
    col = "Eta_average"
    if len(hr_df.loc[~hr_df[col].isna()]) > 0 and len(lr_df.loc[~lr_df[col].isna()]) > 0:
        a = hr_df.loc[~hr_df[col].isna()][col].values
        b = lr_df.loc[~lr_df[col].isna()][col].values
        p_test = scipy.stats.ks_2samp(a,b).pvalue
        all_func_data.append([key, p_test, len(a), len(b), ""])
        
        

In [31]:
gene_df = logrank_df.loc[logrank_df["gene"] == "PALB2"]

for key in gene_df["key"].values:
    
    optimal = logrank_df.loc[logrank_df["key"] == key].iloc[0]
    optimal_breaks = list(map(lambda x : int(float(x)), optimal["breakpoints"].split("|")))
    optimal_high = list(map(lambda x : int(x), optimal["high_regions"].split("|")))
    optimal_low = list(map(lambda x : int(x), optimal["low_regions"].split("|")))
    df = pd.read_csv('../../data/functional_data/PALB2_rodrigue_et_al.csv')
    aa_pos = []
    region = []
    c_pos = []
    for index, row in df.iterrows():
        pos = int(row["variant"][1:-1])
        c_pos.append(pos*3)
        reg = place_position(optimal_breaks, pos*3) + 1
        region.append(reg)

    df["region"] = region
    df["coding_position"] = c_pos
    df = df.drop_duplicates(subset = "coding_position")

    df["in HR"] = df["region"].apply(lambda x : x in optimal_high)
    df["in LR"] = df["region"].apply(lambda x : x in optimal_low)

    hr_df = df.loc[df["in HR"]]
    lr_df = df.loc[df["in LR"]]
    col_of_interest = "Relative % survival"
    to_return = {
        "functional pval" : None,
        "HR covered" : len(hr_df.loc[~hr_df[col_of_interest].isna()]) ,
        "LR covered" : len(lr_df.loc[~lr_df[col_of_interest].isna()]),
        "correlation" : None,
        "correlation p" : None
    }
    if len(hr_df) > 0 and len(lr_df) > 0:
        a = hr_df.loc[~hr_df[col_of_interest].isna()][col_of_interest].values
        b = lr_df.loc[~lr_df[col_of_interest].isna()][col_of_interest].values
        p_test = scipy.stats.ks_2samp(a,b).pvalue
        all_func_data.append([key, p_test, len(a), len(b), ""])

In [32]:

gene_df = logrank_df.loc[logrank_df["gene"] == "TP53"]

for key in gene_df["key"].values:
    df_a = pd.read_excel("../../data/functional_data/TP53_Giacomelli_et_al.xlsx", skiprows = 1)
    df = copy.deepcopy(df_a)
    optimal = logrank_df.loc[logrank_df["key"] == key].iloc[0]
    optimal_breaks = list(map(lambda x : int(float(x)), optimal["breakpoints"].split("|")))
    optimal_high = list(map(lambda x : int(x), optimal["high_regions"].split("|")))
    optimal_low = list(map(lambda x : int(x), optimal["low_regions"].split("|")))
    df["coding_pos"] = df["Position"].apply(lambda x : x*3)
    df = df.drop_duplicates(subset = "coding_pos")
    reg = []
    for index, row in df.iterrows():
        p = row["coding_pos"]
        region = place_position(optimal_breaks, p) + 1
        reg.append(region)

    df["region"] = reg
    df["in HR"] = df["region"].apply(lambda x : x in optimal_high)
    df["in LR"] = df["region"].apply(lambda x : x in optimal_low)
    hr_df = df.loc[df["in HR"]]
    lr_df = df.loc[df["in LR"]]
    col = "A549_p53NULL_Nutlin-3_Z-score"


    if len(hr_df.loc[~hr_df[col].isna()]) > 0 and len(lr_df.loc[~lr_df[col].isna()]) > 0:
        a = hr_df.loc[~hr_df[col].isna()][col].values
        b = lr_df.loc[~lr_df[col].isna()][col].values
        p_test = scipy.stats.ks_2samp(a,b).pvalue
        all_func_data.append([key, p_test, len(a), len(b), ""])
        
        

In [33]:
gene_df = logrank_df.loc[logrank_df["gene"] == "CHEK2"]

for key in gene_df["key"].values:
    optimal = logrank_df.loc[logrank_df["key"] == key].iloc[0]
    breaks = optimal["breaks"]
    optimal_breaks = list(map(lambda x : int(float(x)), optimal["breakpoints"].split("|")))
    optimal_high = list(map(lambda x : int(x), optimal["high_regions"].split("|")))
    optimal_low = list(map(lambda x : int(x), optimal["low_regions"].split("|")))

    df = pd.read_csv("../../data/functional_data/CHEK2_Delimitsou_et_al.csv")
    df = df.rename(columns = {"In vivo\xa0results -Characterization" : "In vivo results -Characterization"})
    pos = []
    region = []
    for index, row in df.iterrows():
        c_str = row["cDNA (NM_007194.3)"]
        if ">" not in c_str:
            pos.append("N/A")
            region.append("N/A")
        else:
            c_pos = int(c_str.split(">")[0][2:-1])
            pos.append(c_pos)

            reg = place_position(optimal_breaks, c_pos) + 1
            region.append(reg)

    df["c_pos"] = pos
    df["region"] = region
    df["in HR"] = df["region"].apply(lambda x : x in optimal_high)
    df["in LR"] = df["region"].apply(lambda x : x in optimal_low)
    hr_df = df.loc[df["in HR"]]
    lr_df = df.loc[df["in LR"]]

    col = "In vivo results -Characterization"

    to_return = {
        "functional pval" : None,
        "HR covered" : len(hr_df.loc[~hr_df[col].isna()]) ,
        "LR covered" : len(lr_df.loc[~lr_df[col].isna()]),
        "correlation" : None,
        "correlatipn p" : None
    }
    if len(hr_df) > 0 and len(lr_df) > 0:
        a = len(hr_df.loc[hr_df[col] == "Benign"])
        b = len(hr_df.loc[hr_df[col] == "Damaging"])
        c = len(lr_df.loc[lr_df[col] == "Benign"])
        d = len(lr_df.loc[lr_df[col] == "Damaging"])

        functional_reports = [[a,b],[c,d]]
        p_test = scipy.stats.chi2_contingency(functional_reports)[1]
        all_func_data.append([key, p_test, a+b,c+d, str(a) + "|" + str(b) + "|" + str(c) + "|" + str(d)])
        
        

In [34]:
gene_df = logrank_df.loc[logrank_df["gene"] == "MSH6"]

for key in gene_df["key"].values:    
    
    df_a = pd.read_excel('../../data/functional_data/MSH6_ijms-1268836-supplementary.xlsx', sheet_name = "MMR activity")
    df_a = df_a.loc[df_a["Nomenclature protein"].str.contains("p.")]
    p_pos = []

    for index, row in df_a.iterrows():
        pieces = row["Nomenclature protein"].split('p.')[1]
        try:
            p_integer = int(pieces[4:-4])
            p_pos.append(p_integer)
        except:
            p_pos.append(None)

    df_a["p_pos"] = p_pos
    df_a = df_a.loc[~df_a["p_pos"].isna()]
    df_a["c_pos"] = df_a["p_pos"].apply(lambda x : x * 3)

    df = copy.deepcopy(df_a)
    optimal = logrank_df.loc[logrank_df["key"] == key].iloc[0]
    breaks = optimal["breaks"]
    optimal_breaks = list(map(lambda x : int(float(x)), optimal["breakpoints"].split("|")))
    optimal_high = list(map(lambda x : int(x), optimal["high_regions"].split("|")))
    optimal_low = list(map(lambda x : int(x), optimal["low_regions"].split("|")))

    reg = []
    for index, row in df.iterrows():
        p = row["c_pos"]
        region = place_position(optimal_breaks, p) + 1
        reg.append(region)

    df["region"] = reg
    df["in HR"] = df["region"].apply(lambda x : x in optimal_high)
    df["in LR"] = df["region"].apply(lambda x : x in optimal_low)
    hr_df = df.loc[df["in HR"]]
    lr_df = df.loc[df["in LR"]]

    a = len(hr_df.loc[hr_df["Result"] != "MMR proficient"])
    b = len(hr_df.loc[hr_df["Result"] == "MMR proficient"])
    c = len(lr_df.loc[lr_df["Result"] != "MMR proficient"])
    d = len(lr_df.loc[lr_df["Result"] == "MMR proficient"])
    functional_reports = [[a,b],[c,d]]
    p = scipy.stats.chi2_contingency(functional_reports)[1]
    all_func_data.append([key, p, a+b, c+d, str(a) + "|" + str(b) + "|" + str(c) + "|" + str(d)])
    
    

In [35]:
df = pd.DataFrame(all_func_data, columns = ["key", "p_value", "HR Reports", "LR Reports", "Contingency Table"])
df.to_csv("010422_functional_vals_all_keys.csv")